# Image classification with Keras

![](http://)## Install dependencies

In [80]:
%matplotlib inline
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from scipy.ndimage.filters import gaussian_filter
from scipy import ndimage

from keras import regularizers, optimizers
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import (
    random_rotation, random_shift, random_shear, random_zoom,
    random_channel_shift, img_to_array, ImageDataGenerator)
from keras import backend as K
from keras.datasets import fashion_mnist
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
print("import is ready")

import is ready


In [81]:
# set consistent random seed
random_seed = 2018
np.random.seed(random_seed)  
tf.set_random_seed(random_seed)

## Dataset
Whales dataset

### Show the content of the current folder

In [82]:
print(os.listdir(".."))
print(os.listdir("."))

['results.csv', 'lib', 'working', 'config', 'input', 'results_pred.csv']
['train', 'test', 'train.csv', 'sample_submission.csv']


### Show the content of the input folder

In [83]:
print(os.listdir("train"))

['72c3ce75c.jpg', 'a7ad640ee.jpg', 'df2b6c364.jpg', '26013fcb5.jpg', '09eff7b37.jpg', '321f922d3.jpg', 'a08161913.jpg', 'd2dde23b0.jpg', '111d1ab30.jpg', 'cec27e4e3.jpg', '89acede28.jpg', 'b592e25d4.jpg', '621c3ed03.jpg', '86517d536.jpg', 'c5f4b73be.jpg', '5afaf2ffd.jpg', 'b060f5529.jpg', '7919aa309.jpg', '5a35f6ded.jpg', 'f25a1c3f5.jpg', '8d4453e81.jpg', '3b2468366.jpg', '929b9b616.jpg', 'e2deae874.jpg', '6802020c8.jpg', '1b5501c54.jpg', '54cfc2e29.jpg', '935851f78.jpg', '2f794634f.jpg', '9659257b6.jpg', '9d83a3f97.jpg', '6516fdf9a.jpg', '50e48ba76.jpg', '7b65d88a1.jpg', '430e59cfe.jpg', '8aec615b2.jpg', 'f9832ec6b.jpg', '8ec227a2d.jpg', '6325cd7bc.jpg', '773996d67.jpg', '8121b306c.jpg', '4ef622128.jpg', '417a39ef8.jpg', '129776139.jpg', '77b7e53cd.jpg', '42fdde329.jpg', '915f52b29.jpg', '4bfb77747.jpg', '0f61f7349.jpg', '439867f0f.jpg', '83ba3bdea.jpg', '93d091bc1.jpg', '476c2eb33.jpg', 'd36d43a20.jpg', 'a52530551.jpg', '3cffa45c3.jpg', '30313cca2.jpg', '02dfe3c04.jpg', '7971a68ba.jp

### Importing, normalizing, visualizing

Let's upload whales dataset.

In [84]:
# flow_from_dataframe
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
traindf=pd.read_csv("../input/train.csv",dtype=str)
testdf=pd.read_csv("../input/sample_submission.csv",dtype=str)

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

Print obtained dataframes for checking

In [85]:
traindf.head(1)

,Image,Id
0,0000e88ab.jpg,w_f48451c


In [86]:
testdf.head(1)

,Image,Id
0,00028a005.jpg,new_whale w_23a388d w_9b5109b w_9c506f6 w_0369a5c


In [87]:
# Pass the dataframes to 2 different flow_from_dataframe functions
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="./train/",
x_col="Image",
y_col="Id",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="./train/",
x_col="Image",
y_col="Id",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./test/",
x_col="Image",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))


Found 19021 images belonging to 5005 classes.
Found 6340 images belonging to 5005 classes.
Found 7960 images.


In [88]:
# Model @frommedium
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
# model.add(Dense(8192))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5005, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

print("The model is ready")


The model is ready


In [89]:
# Fit the model @frommedium
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1
)

Epoch 1/1
594/594 [==============================] - 220s 371ms/step - loss: 6.0037 - acc: 0.3772 - val_loss: 6.3003 - val_acc: 0.3889


In [90]:
# Evaluate model
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
model.evaluate_generator(generator=valid_generator, steps=1)

[6.302807807922363, 0.40625]

In [91]:
print(os.listdir("."))

['train', 'test', 'train.csv', 'sample_submission.csv']


In [92]:
# Predict the output
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames=test_generator.filenames

print("filenames were prepared")

248/248 [==============================] - 70s 282ms/step
filenames were prepared


In [93]:
print(os.listdir("."))
results=pd.DataFrame({"Filename":filenames})
results_pred=pd.DataFrame({"Predictions":predictions})
results.to_csv("../results.csv",index=False)
results_pred.to_csv("../results_pred.csv",index=False)
print(os.listdir(".."))
print("results were written to the output")

['train', 'test', 'train.csv', 'sample_submission.csv']
['results.csv', 'lib', 'working', 'config', 'input', 'results_pred.csv']
results were written to the output
